In [1]:
import sys
sys.path.append("../src/")

In [2]:
%load_ext autoreload
%autoreload 2
import data_utils as du

In [3]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  


In [4]:
data = du.get_wiki_data(5)

INFO:absl:No config specified, defaulting to first: wiki40b/en
INFO:absl:Load dataset info from gs://tfds-data/datasets/wiki40b/en/1.3.0
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.splits from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.module_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset wiki40b (gs://tfds-data/datasets/wiki40b/en/1.3.0)
INFO:absl:Constructing tf.data.Dataset for split train[:5], from gs://tfds-data/datasets/wiki40b/en/1.3.0


# Given 1 document

* split doc into paragraphs
* filter paragraph by length
* encode each paragraph
* serialize output

In [9]:
du.get_paragraphs(data[1])

['\nKasturba Road is a street in Bangalore, the capital of Karnataka, India, which is connected to M G Road to the north and J C Road to the south. Some important landmarks situated along Kasturba Road are Sree Kanteerava Stadium, Kanteerava Indoor Stadium, Cubbon Park, Government Museum, Venkatappa Art Gallery, Visvesvaraya Industrial and Technological Museum and UB City. A 600-year-old Ganesha temple is also situated on Kasturba Road.\nIt was earlier known as Sydney Road.\nOther important landmarks close to the road are Karnataka High Court, Vidhana Soudha and Chinnaswamy Stadium.']

In [67]:
reprs = du.srl_paragraphs_batched(du.get_paragraphs(data[1]))

In [64]:
# Use cPickle
du.serialize_reprs(du.srl_paragraphs_batched(du.get_paragraphs(data[1])), "/tmp/test.pkl")

In [66]:
import pickle
with open("/tmp/test.pkl", "rb") as f:
    unpickler = pickle.Unpickler(f)
    test = unpickler.load() 
print(test)

('Kasturba Road is a street in Bangalore, the capital of Karnataka, India, which is connected to M G Road to the north and J C Road to the south.', [[('V', [6]), ('ARG1', [1, 2, 3, 4, 5]), ('ARG2', [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])], [('V', [20])], [('V', [21]), ('ARG1', [7, 8, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])]])


# Parallelize one worker
* batchify over multiple paragraphs
* write all outputs to single file

# shard data
* simply take a parameter for what slice of data to operate over
* check all pylint bugs

# Benchmark
* time execution of 1 paragraph on cpu vs gpu
* on best setup, optimize batch size
* do calculation and choose appropriate sharing

In [ ]:
paragraphs = []
for d in data:
    text = d['text'].numpy().decode('utf-8')
    for para in text.split("_START_"):
        if not para.startswith("PARAGRAPH_"):
            continue
        proced_text = du.preproc_text(para)
        if len(proced_text) < 500:
            continue
        paragraphs.append(proced_text)
        # print(proced_text)
        # print(len(proced_text))
        
        #print("~"*20)
    #print("#"*20)
print(len(paragraphs))

In [ ]:
print(paragraphs[100])

# Format
* We need to keep both paragraph level and document level examples together.
* We can train consecutively any paragraph but also we can train any document with multiple paragraphs

## First try
* Split by paragraph
* Filter for paragraph length (above some length)
* We get roughly 1 paragraph over 1000 characters per document. That's not great.
* 789 / 500 docs with cutoff at 500 characters

# TODO
* Upgrade Spacy en_core_web_sm to latest
* fix data encoder error

In [ ]:
reprs = du.encode_data_srl_only(data[:10])

In [ ]:
import sys

In [ ]:
sys.getsizeof(data[:10])

In [ ]:
sys.getsizeof(reprs)

In [ ]:
len(reprs)

In [ ]:
reprs[0]

In [ ]:
from collections import defaultdict



In [ ]:
def encode_data_srl_only(data):
    nlp = du.create_nlp()
    tokenizer = du.BertTokenizer.from_pretrained('bert-base-uncased')
    predictor = du.make_allennlp()
    reprs = []
    for ex_idx, d in enumerate(data):
        text = d['text'].numpy().decode('UTF-8').encode('ascii', 'ignore').decode('UTF-8')
        for para in text.split("_START_"):
            if not para.startswith("PARAGRAPH_"):
                continue
            example = du.preproc_text(para)
            if len(example) < 500:
                continue
            doc = nlp(example)
            for sent in doc.sents:
                span = sent.text
                lm_tokenized_input = tokenizer(span, return_tensors='pt')
                srl_parse = predictor.predict(sentence=span)
                token_strs = tokenizer.convert_ids_to_tokens(lm_tokenized_input['input_ids'].numpy()[0])
                try:
                    word2veclist = new_map_words_to_vectors(srl_parse['words'], token_strs)
                except du.NoStopException as e:
                    continue
                except Exception as e:
                    print("Index", ex_idx)
                    print(e)
                    print(token_strs)
                    print(srl_parse['words'])
                    raise e
                sent_repr = build_repr(srl_parse['verbs'], word2veclist)
                reprs.append((span, sent_repr))
    return reprs

In [ ]:
reprs = encode_data_srl_only(data[:2])
print(reprs)

In [ ]:
reprs[0]

In [ ]:
tokenizer = du.BertTokenizer.from_pretrained('bert-base-uncased')
span = '\nClayton was born in Mobile in south Alabama, but he was reared in Alexandria, Virginia.'
lm_tokenized_input = tokenizer(span, return_tensors='pt')


In [ ]:
lm_tokenized_input

# Fix parsing errors
* fix parser
* update spacy

In [ ]:
data = du.get_wiki_data(500)

In [ ]:
reprs = encode_data_srl_only(data[27:])

In [ ]:
reprs[33]

In [ ]:
predictor = du.make_allennlp()

In [ ]:
predictor.predict(sentence='\nClayton was born in Mobile in south Alabama, but he was reared in Alexandria, Virginia.'
)

In [ ]:
for ex_idx, d in enumerate(data[:1]):
    text = d['text'].numpy().decode('utf-8')
    for para in text.split("_START_"):
        if not para.startswith("PARAGRAPH_"):
            continue
        example = du.preproc_text(para)
        if len(example) < 500:
            continue
        doc = nlp(example)
        for sent in doc.sents:
            span = sent.text
            lm_tokenized_input = tokenizer(span, return_tensors='pt')
            srl_parse = predictor.predict(sentence=span)
            token_strs = tokenizer.convert_ids_to_tokens(lm_tokenized_input['input_ids'].numpy()[0])
            word2veclist = new_map_words_to_vectors(srl_parse['words'], token_strs)

In [ ]:
data[6]['text'].numpy().decode('UTF-8').encode('ascii', 'ignore').decode('UTF-8').split("_START_")# .decode("utf-8")

In [ ]:
type(data[6]['text'].numpy())